# Ch10

In [1]:
import pandas as pd 
import numpy as np 

In [2]:
from scipy.stats import norm

In [20]:
norm.cdf(1)

0.8413447460685429

In [ ]:
xx = np.linspace(-8, 8, 100)
cdf = rv.cdf(xx)
plt.plot(xx, cdf)
plt.title("누적분포함수 ")
plt.xlabel("$x$")
plt.ylabel("$F(x)$")
plt.show()

## 10.2 Strategy-Independent Bet Sizing Approaches

* C t,l / C t,s : Long / Short 시그널이 여러번 나올 수 있음을 가정(하나의 전략에서 Multiple Signal or Multiple Strategy) 

F[Ct] - F[0]

A second solution is to follow a budgeting approach. 

A third approach is to apply meta-labeling, as we explained in Chapter 3. We fit a classifier, such as an SVC or RF, to determine the probability of misclassification, and use that probability to derive the bet size. This approach has a couple of advantages: First, the ML algorithm that decides the bet sizes is independent of the primary model, allowing for the incorporation fo features predictive of false positives. Second, the predicted probability can be directly translated into bet size. 

## 10.3 Bet Sizing From Predicted Probabilities

In [28]:
A = np.linspace(0.5, 0.99, 10)
A

array([0.5       , 0.55444444, 0.60888889, 0.66333333, 0.71777778,
       0.77222222, 0.82666667, 0.88111111, 0.93555556, 0.99      ])

In [35]:
from scipy.stats import norm
from adv_finance.multiprocess import mp_pandas_obj


def discrete_signal(signal, step_size):
    # Discrete signal
    disc_sig = (signal / step_size).round() * step_size
    disc_sig[disc_sig > 1] = 1
    disc_sig[disc_sig < -1] = -1
    return disc_sig

def get_signal(events, step_size, prob, pred, num_classes, num_threads, **kwargs):
    # Get signals from predictions
    if prob.shape[0] == 0:
        return pd.Series()
    # Generate signals from multinomial
    signal0 = (prob - 1. / num_classes) / np.sqrt(prob * (1. - prob))
    signal0 = pred * (2 * norm.cdf(signal0) - 1)
    if 'side' in events:
        signal0 *= events.loc[signal0.index, 'side']
    # Averaging
    df0 = signal0.to_frame('signal').join(events[['t1']], how='left')
    df0 = avg_active_signals(df0, num_threads)
    signal1 = discrete_signal(signal0=df0, step_size=step_size)
    return signal1

def mp_avg_active_signals(signals, molecule):
    out = pd.Series()
    for loc in molecule:
        is_act = (signals.index.values <= loc) & ((loc < signal['t1']) | pd.isnull(signals['t1']))
        act = signals[is_act].index
        if len(act) > 0:
            out[loc] = signals.loc[act, 'signal'].mean()
        else:
            out[loc] = 0
    return out

def avg_active_signals(signals, num_threads):
    # Compute the average signal
    # 1) time points where singal changes
    t_pnts = set(signals['t1'].dropna().values)
    t_pnts = t_pnts.union(signals.index.values)
    t_pnts = list(t_pnts)
    t_pnts.sort();
    out = mp_pandas_obj(mp_avg_active_signals, ('molecule', t_pnts), num_threads, signals=signals)
    return out